In [35]:
#
#     The data used in this notebook was cleaned by the script by MadScientist:
#     https://www.kaggle.com/keremt/very-extensive-cleaning-by-sberbank-discussions
#
#

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error

def preprocess(data):
    #add relative floor
    data['rel_floor'] = data['floor']/data['max_floor']
    #add relative kitch_sq
    data['relative_kitch_sq'] = data['kitch_sq']/data['full_sq']
    #add room size
    data['room_size'] = data['life_sq']/data['num_room']
    # add month and day of week
    data['month'] = data.timestamp.dt.month.astype(int)
    data['day_of_week'] = data.timestamp.dt.dayofweek.astype(int)
    
    data['bought_minus_built'] = data.timestamp.dt.year.astype(int) - data['build_year']
    
    data.loc[data['full_sq']==data['life_sq'],'life_sq'] = np.nan
    return data
def fact_binary(data):
    data = data.applymap(lambda x: x if x!='yes' else 1)
    data = data.applymap(lambda x: x if x!='no' else 0)
    data = data.applymap(lambda x: x if x!='OwnerOccupier' else 0)
    data = data.applymap(lambda x: x if x!='Investment' else 1)
    return data
class sep_estimator:
    def __init__(self,owner_params,investment_params):
        self.est1 = xgb.XGBRegressor(**owner_params)
        self.est2 = xgb.XGBRegressor(**investment_params)
    def preprocess_owner(self,data,mode):
        assert (data['product_type'].values==0).all()
        if mode=='predict':
            data.loc[data['full_sq'].isnull(),'full_sq'] = 50
        data = data.drop('timestamp',axis=1)
        return data
    def preprocess_investment(self,data,mode):
        if mode=='predict':
            assert (data['product_type'].values==1).all()
            data.loc[data['full_sq'].isnull(),'full_sq'] = 50
        data = data.drop('timestamp',axis=1)
        return data
    def fit(self,X,y):
        X1 = X[X['product_type']==0]
        X2 = X
        X1 = self.preprocess_owner(X1,'train')
        y1 = y.loc[X1.index.values]/X1['full_sq']
        X2 = self.preprocess_investment(X2,'train')
        y2 = y
        y2 = y2/X2['full_sq']
        if len(X1)>0:
            self.est1.fit(X1,y1)
        if len(X2)>0:
            self.est2.fit(X2,y2)
    def predict(self,X):
        X1 = X[X['product_type']==0]
        X2 = X[X['product_type']==1]
        owner_index = X1.index.values
        investment_index = X.index.drop(owner_index).values
        X1 = self.preprocess_owner(X1,'predict')
        X2 = self.preprocess_investment(X2,'predict')
        res = pd.DataFrame(index=X.index)
        if len(X1)>0:
            pred1 = self.est1.predict(X1)
            res.loc[owner_index,0] = pred1*X1['full_sq']
        if len(X2)>0:
            pred2 = self.est2.predict(X2)
            res.loc[investment_index,0] = pred2*X2['full_sq']     
        return res[0].values.flatten()

In [2]:
train = pd.read_csv('../data-corr-clean/foo.csv',index_col='id',parse_dates=['timestamp'])

In [27]:
def prepare_train(train1):
    train = train1.copy()
    train = fact_binary(train)
    train = preprocess(train)
    train = train.drop(['sub_area','ecology'],axis=1)
    train = train.loc[~train['full_sq'].isnull(),:]
    #
    #    Price level multipliers here. I've copied it from Andy Harless script:
    #    https://www.kaggle.com/aharless/exercising-the-exorcism
    #
    #

    rate_2015_q2 = 1
    rate_2015_q1 = rate_2015_q2 / 0.9932
    rate_2014_q4 = rate_2015_q1 / 1.0112
    rate_2014_q3 = rate_2014_q4 / 1.0169
    rate_2014_q2 = rate_2014_q3 / 1.0086
    rate_2014_q1 = rate_2014_q2 / 1.0126
    rate_2013_q4 = rate_2014_q1 / 0.9902
    rate_2013_q3 = rate_2013_q4 / 1.0041
    rate_2013_q2 = rate_2013_q3 / 1.0044
    rate_2013_q1 = rate_2013_q2 / 1.0104  # This is 1.002 (relative to mult), close to 1:
    rate_2012_q4 = rate_2013_q1 / 0.9832  #     maybe use 2013q1 as a base quarter and get rid of mult?
    rate_2012_q3 = rate_2012_q4 / 1.0277
    rate_2012_q2 = rate_2012_q3 / 1.0279
    rate_2012_q1 = rate_2012_q2 / 1.0279
    rate_2011_q4 = rate_2012_q1 / 1.076
    rate_2011_q3 = rate_2011_q4 / 1.0236
    rate_2011_q2 = rate_2011_q3 / 1
    rate_2011_q1 = rate_2011_q2 / 1.011


    # train 2015
    train['average_q_price'] = 1

    train_2015_q2_index = train.loc[train['timestamp'].dt.year == 2015].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
    train.loc[train_2015_q2_index, 'average_q_price'] = rate_2015_q2

    train_2015_q1_index = train.loc[train['timestamp'].dt.year == 2015].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
    train.loc[train_2015_q1_index, 'average_q_price'] = rate_2015_q1


    # train 2014
    train_2014_q4_index = train.loc[train['timestamp'].dt.year == 2014].loc[train['timestamp'].dt.month >= 10].loc[train['timestamp'].dt.month <= 12].index
    train.loc[train_2014_q4_index, 'average_q_price'] = rate_2014_q4

    train_2014_q3_index = train.loc[train['timestamp'].dt.year == 2014].loc[train['timestamp'].dt.month >= 7].loc[train['timestamp'].dt.month < 10].index
    train.loc[train_2014_q3_index, 'average_q_price'] = rate_2014_q3

    train_2014_q2_index = train.loc[train['timestamp'].dt.year == 2014].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
    train.loc[train_2014_q2_index, 'average_q_price'] = rate_2014_q2

    train_2014_q1_index = train.loc[train['timestamp'].dt.year == 2014].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
    train.loc[train_2014_q1_index, 'average_q_price'] = rate_2014_q1


    # train 2013
    train_2013_q4_index = train.loc[train['timestamp'].dt.year == 2013].loc[train['timestamp'].dt.month >= 10].loc[train['timestamp'].dt.month <= 12].index
    train.loc[train_2013_q4_index, 'average_q_price'] = rate_2013_q4

    train_2013_q3_index = train.loc[train['timestamp'].dt.year == 2013].loc[train['timestamp'].dt.month >= 7].loc[train['timestamp'].dt.month < 10].index
    train.loc[train_2013_q3_index, 'average_q_price'] = rate_2013_q3

    train_2013_q2_index = train.loc[train['timestamp'].dt.year == 2013].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
    train.loc[train_2013_q2_index, 'average_q_price'] = rate_2013_q2

    train_2013_q1_index = train.loc[train['timestamp'].dt.year == 2013].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
    train.loc[train_2013_q1_index, 'average_q_price'] = rate_2013_q1


    # train 2012
    train_2012_q4_index = train.loc[train['timestamp'].dt.year == 2012].loc[train['timestamp'].dt.month >= 10].loc[train['timestamp'].dt.month <= 12].index
    train.loc[train_2012_q4_index, 'average_q_price'] = rate_2012_q4

    train_2012_q3_index = train.loc[train['timestamp'].dt.year == 2012].loc[train['timestamp'].dt.month >= 7].loc[train['timestamp'].dt.month < 10].index
    train.loc[train_2012_q3_index, 'average_q_price'] = rate_2012_q3

    train_2012_q2_index = train.loc[train['timestamp'].dt.year == 2012].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
    train.loc[train_2012_q2_index, 'average_q_price'] = rate_2012_q2

    train_2012_q1_index = train.loc[train['timestamp'].dt.year == 2012].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
    train.loc[train_2012_q1_index, 'average_q_price'] = rate_2012_q1


    # train 2011
    train_2011_q4_index = train.loc[train['timestamp'].dt.year == 2011].loc[train['timestamp'].dt.month >= 10].loc[train['timestamp'].dt.month <= 12].index
    train.loc[train_2011_q4_index, 'average_q_price'] = rate_2011_q4

    train_2011_q3_index = train.loc[train['timestamp'].dt.year == 2011].loc[train['timestamp'].dt.month >= 7].loc[train['timestamp'].dt.month < 10].index
    train.loc[train_2011_q3_index, 'average_q_price'] = rate_2011_q3

    train_2011_q2_index = train.loc[train['timestamp'].dt.year == 2011].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
    train.loc[train_2011_q2_index, 'average_q_price'] = rate_2011_q2

    train_2011_q1_index = train.loc[train['timestamp'].dt.year == 2011].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
    train.loc[train_2011_q1_index, 'average_q_price'] = rate_2011_q1

    train['price_doc'] = train['price_doc'] * train['average_q_price']


    #########################################################################################################


    X = train.drop(["price_doc", "average_q_price"],axis=1)
    y = train['price_doc']
    return X,y

In [40]:
estimators = []
bar = {}
kf = KFold(n_splits = 5, random_state = 3228, shuffle = True)
for train_index, test_index in kf.split(train):
    train_X, train_y = prepare_train(train.iloc[train_index])
    test, test_y = prepare_test(train.iloc[test_index])
    
    owner_params = {
            'n_estimators':100,
            'learning_rate':0.1,
            'max_depth':7,
            'min_child_weight':1,
            'subsample':0.8,
            'colsample_bytree':0.9,
            'colsample_bylevel':1,
            'reg_alpha':0,
            'reg_lambda':1,
            'seed':0,
            'objective':'reg:linear',
            'nthread':8
    }
    investment_params = {
            'n_estimators':100,
            'learning_rate':0.1,
            'max_depth':7,
            'min_child_weight':1,
            'subsample':0.8,
            'colsample_bytree':0.9,
            'colsample_bylevel':1,
            'reg_alpha':0,
            'reg_lambda':1,
            'seed':0,
            'objective':'reg:linear',
            'nthread':8
    }
    est = sep_estimator(owner_params=owner_params,investment_params=investment_params)
    est.fit(train_X, train_y)
    estimators.append(est)
    bar[est] = test
    pr = est.predict(test)
    print(mean_squared_log_error(test_y, pr))
#     pr和test_y比较

/Users/gww/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


0.40900889697960735


/Users/gww/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


0.4631123159852099


/Users/gww/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


0.5250402741696152


/Users/gww/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


0.6433835750914071
0.5009248538478593


/Users/gww/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [33]:
for (k,v) in bar.items():
    pred_test = np.zeros(shape=v.values.shape[0])
    cnt = 0
    pr = k.predict(v)
    pred_test += pr
    print(pr)
    pred_test /= len(estimators)
    pred_test *= 0.9915
#     subm = pd.read_csv('sample_submission.csv')
#     subm['price_doc'] = pred_test
#     subm.to_csv('model_1_output.csv',index=False)
#     subm.head()
    

/Users/gww/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


[ 3307106.15625     4473379.96875     5036297.25        2827337.25
  7917182.4921875   4681291.1640625   3601198.4765625   5254565.765625
  4310872.28125     2720344.34375     5266751.5625      5719595.578125
  3333408.90625     2559763.046875    3593831.4140625   5109194.7421875
  3210250.46875     3775752.09375     3177805.46875     5963941.53125
  3593657.4765625   5480601.328125    5318868.8984375   2836740.0625
 13741373.1875      4243700.171875    3625384.5859375   4929474.59375
  2785272.36328125  3966680.6796875   1679241.0078125   6702468.671875
  1618715.9296875   5429946.484375    9705829.6875      3160909.984375
 13790248.59375     6901880.578125    4144951.234375    4749227.5
  3764121.40625     3942502.984375    4612001.7265625   9614644.921875
  3570186.65625     3248790.46875     4259754.375       4465959.99609375
  7651056.2734375  10586615.953125    4575352.296875    6270947.890625
  5813119.46875     4717948.21875     4733527.1875      7720880.2734375
  8826235.875  

[ 4933258.6484375   4542150.3125      4486652.765625    4079170.578125
   973662.94921875  6039946.40625     2857087.734375    3613843.9375
  2958717.7421875   3580518.046875    5052206.875       3938655.65625
  3465734.90625     4610302.125       3173100.125       3629097.578125
  9935598.75        5248986.4375      4481451.9140625   6863049.703125
  2175580.1953125   4720477.          4957899.984375    5309858.59375
  3534274.765625    9688989.75        4529967.5390625   6284738.0625
  7823216.40625     5178323.140625    4996606.65625     6133406.625
  6766663.265625    4049670.234375    3546531.75        5345068.9453125
  4938564.125       6391043.796875    6570866.6953125   3628634.53125
  4888069.5703125   3749353.125       6803905.625       6016913.296875
  6700139.0625      3556041.5         4677426.09375     4847141.9296875
  4911853.03125     2010890.8203125   8755620.65625     8174848.125
  4815324.4296875   4029681.0390625   5511392.984375    5937674.625
  3776643.140625    

In [34]:
len(pred_test)

200

In [14]:
print(pred_test)

[1699031.38676953 2821311.44980781 1304456.49298594  989070.9489375
  657721.17973125  816344.36760938 1252360.04186953  918540.41672813
  959794.22199375 1000205.18409375 1308197.67810703  846110.38740938
  764558.20641797  634748.44696875 1697152.73827148  671403.64270078
  941327.62125     933944.0446875   773110.078275   1432766.86430625
  840144.35994609 1207384.51896563 1046959.34490469 1166809.9159875
 1228541.86790156  778883.62615313  661991.17440586 2284417.25176875
  787051.51949531 1043356.14714844  724656.19985859  583554.6373125
 1046875.13557031  977349.1880625   740723.4744     1131849.08995781
  508174.19524687  383746.38920156 1410569.97099844 2426268.20860547
  740134.0524375   922548.13409063  820535.37768984  860446.88096016
 2211905.12625     629320.467825   1240923.01965469 1111144.38246094
 1446152.56512891  420758.51408906 1048734.42115781  828632.26259063
  624511.64325     841146.14830781  880742.50175391  607843.85279062
 3222354.36130781  861174.16635     6

In [39]:

def prepare_test(test):
    #test1 = kaggle_test.copy()
    test1 = test.copy()
    test1 = preprocess(test1)
    test1 = fact_binary(test1)
    #test1 = test1.merge(macro,on='timestamp')
    test_y = test1['price_doc']
    test1 = test1.drop(['ecology','sub_area','price_doc'],axis=1)
    test1.loc[test1['full_sq'].isnull(),'full_sq'] = 50
    test1.loc[test1['product_type'].isnull(),'product_type'] = 1
    return test1, test_y

In [ ]:
estimators = []
for i in range(400):
    owner_params = {
            'n_estimators':100,
            'learning_rate':0.1,
            'max_depth':7,
            'min_child_weight':1,
            'subsample':0.8,
            'colsample_bytree':0.9,
            'colsample_bylevel':1,
            'reg_alpha':0,
            'reg_lambda':1,
            'seed':i,
            'objective':'reg:linear',
            'nthread':8
    }
    investment_params = {
            'n_estimators':100,
            'learning_rate':0.1,
            'max_depth':7,
            'min_child_weight':1,
            'subsample':0.8,
            'colsample_bytree':0.9,
            'colsample_bylevel':1,
            'reg_alpha':0,
            'reg_lambda':1,
            'seed':i,
            'objective':'reg:linear',
            'nthread':8
    }
    est = sep_estimator(owner_params=owner_params,investment_params=investment_params)
    est.fit(X,y)
    estimators.append(est)
    if i%10==0:
        print i


In [ ]:
pred_test = np.zeros(shape=test1.values.shape[0])
cnt = 0
for e1 in estimators:
    pr = e1.predict(test1)
    pred_test += pr
    if cnt%10 == 0:
        print cnt
    cnt += 1
pred_test /= len(estimators)
pred_test *= 0.9915
subm = pd.read_csv('sample_submission.csv')
subm['price_doc'] = pred_test
subm.to_csv('model_1_output.csv',index=False)
subm.head()